In [1]:
import sys
from timeit import default_timer as timer
import pmm.inputs
import pmm.pmm
import pmm.conversions
import MDAnalysis as mda
from MDAnalysis.analysis import align
from MDAnalysis.analysis.rms import rmsd
import numpy as np
#import numpy.linalg 
import nglview as nv

In [8]:
# gather the electronic properties
start = timer()
gauss_file = 'water/qm/wat_esp{}.log'
n_el_states = 4
qm_inputs = pmm.inputs.get_tot_input_gauss(gauss_file, n_el_states)
end = timer()
print('inputs da Gaussian ci mette: ', end - start)
#print(qm_inputs)

qc_geom = pmm.pmm.convert2Universe(qm_inputs['geometry'])
cdm_ref = qc_geom.atoms.center_of_mass()
qc_geom.atoms.positions = qc_geom.atoms.positions - cdm_ref
print(qc_geom)

inputs da Gaussian ci mette:  0.010010900004999712
<Universe with 3 atoms>
/mnt/d/miniconda3/envs/pmm/lib/python3.9/site-packages/MDAnalysis/core/universe.py:467: UserWarning: Residues specified but no atom_resindex given.  All atoms will be placed in first Residue.
  warnings.warn(
/mnt/d/miniconda3/envs/pmm/lib/python3.9/site-packages/MDAnalysis/core/universe.py:473: UserWarning: Segments specified but no segment_resindex given.  All residues will be placed in first Segment
  warnings.warn(


In [6]:
# Read the MD trajectory
start = timer()
traj = mda.Universe('water/mm/md.tpr', 'water/mm/trajout.xtc')
end = timer()
print('leggere xtc ci mette: ', end - start)
w = nv.show_mdanalysis(traj)
w

#qc_traj, solv_traj = pmm.split_qc_solv(traj, '1:28')
#print(qc_traj)
#w = nv.show_mdanalysis(solv_traj)
#w

leggere xtc ci mette:  0.07086170000548009


NGLWidget(max_frame=10)

In [9]:
# Let's consider only a single frame
for i, ts in enumerate(traj.trajectory):
    # Divide the trajectory in QC and solvent.
    # NOTE: the indexes for the QC are inclusive of the extremes.
    qc_traj, solv_traj = pmm.pmm.split_qc_solv(traj, '1:3')
    #print(pmm.rotate_dip_matrix(qm_inputs['dip_matrix'], qc_traj, qc_geom))

    # Calculate the center of mass of QC
    cdm = qc_traj.center_of_mass()
    #print(solv_traj.atoms.positions)
    #print('cdm ref ', qc_geom.atoms.center_of_mass())
    # shift origin to the centers of mass of frame_geom to (0, 0, 0).
    # NOTE: it changes the positions in traj_geom permanently.
    qc_traj.atoms.positions = qc_traj.atoms.positions - cdm
    print('cdm traj ', qc_traj.center_of_mass())
    rot_matrix, rmsd = align.rotation_matrix(qc_traj.atoms.positions,
                                             qc_geom.atoms.positions,
                                             weights=qc_geom.atoms.masses)
    #print(rot_matrix)
    if i == 7:
        rot_traj_geom = mda.Universe.empty(qc_geom.atoms.n_atoms, trajectory=True)
        rot_traj_geom.atoms.positions = rot_matrix.dot(qc_traj.atoms.positions.T).T
        print(type(qc_traj), type(qc_geom), type(rot_traj_geom))
        u_tot = mda.Merge(qc_traj, qc_geom.atoms, rot_traj_geom.atoms)
        print(qc_traj.positions)
        print(qc_geom.atoms.positions)
        print(rot_traj_geom.atoms.positions)
        w = nv.show_mdanalysis(u_tot)
w

cdm traj  [-8.65582155e-10 -1.60110255e-09  6.25178970e-10]
cdm traj  [-7.53099409e-11 -1.68066878e-09 -2.79068976e-10]
cdm traj  [-4.68403739e-10  6.06961092e-10 -3.92615541e-10]
cdm traj  [-1.51480678e-10  1.01926827e-09 -1.29914820e-09]
cdm traj  [ 8.60515781e-10 -1.66997893e-10 -5.38832936e-10]
cdm traj  [ 7.56124522e-11 -3.32274192e-10 -1.57684793e-09]
cdm traj  [-7.32701142e-10 -7.93248421e-10  1.29914820e-09]
cdm traj  [-2.06461147e-09  7.97750274e-10  3.94705709e-10]
<class 'MDAnalysis.core.groups.AtomGroup'> <class 'MDAnalysis.core.universe.Universe'> <class 'MDAnalysis.core.universe.Universe'>
[[-0.0621069  -0.00223792  0.01902371]
 [ 0.27789325  0.45776117 -0.800976  ]
 [ 0.70789355 -0.4222399   0.49902374]]
[[ 0.         -0.          0.06649667]
 [ 0.          0.76143    -0.53197336]
 [-0.         -0.76143    -0.53197336]]
[[ 6.0810790e-10  2.3779901e-06  6.4993665e-02]
 [ 6.0810784e-10  8.1381488e-01 -5.1578856e-01]
 [ 6.0810801e-10 -8.1385261e-01 -5.1581824e-01]]
cdm traj

NGLWidget()